In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

import numpy as np
import pandas as pd
!pip install text_hammer
import text_hammer as th
import seaborn as sns
import matplotlib.pyplot as plt
import re
from wordcloud import STOPWORDS
from collections import defaultdict
#%%time
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from transformers import AutoTokenizer,TFBertModel

max_len = 36

import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy,BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy,BinaryAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Dense

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Loading The Data

In [ ]:
import os
new_directory = "/content/sample_data"

# Change the current working directory
os.chdir(new_directory)
current_directory = os.getcwd()
for item in os.listdir(current_directory):
    if os.path.isdir(os.path.join(current_directory, item)):
        print(item)

print(current_directory)

train_data = pd.read_csv('train.csv',usecols=['id','text','target'])
test_data = pd.read_csv('test.csv',usecols=['id','text'])
sample_data = pd.read_csv('sample_submission.csv')

In [ ]:
test_data.head()


In [ ]:
test_data.tail()


In [ ]:
train_data.head()


In [ ]:
train_data.tail()

In [ ]:
train_data.shape


In [ ]:
def preprocess(data_frame, column_name):
    column = column_name
    data_frame[column] = data_frame[column].progress_apply(lambda x: str(x).lower())
    data_frame[column] = data_frame[column].progress_apply(lambda x: th.remove_emails(x))
    data_frame[column] = data_frame[column].progress_apply(lambda x: th.remove_html_tags(x))
    data_frame[column] = data_frame[column].progress_apply(lambda x: th.remove_special_chars(x))
    data_frame[column] = data_frame[column].progress_apply(lambda x: th.remove_accented_chars(x))
    return data_frame






In [ ]:
train_cleaned_data = preprocess(train_data,'text')


In [ ]:
train_cleaned_data[train_cleaned_data.target == 0]


In [ ]:
train_data = train_cleaned_data.copy()


In [ ]:
train_data.head(10)


* Target 1 indicates any accident or disaster
* Target 0 indicates a no attetntion


### Word Cloud:

In [ ]:
import nltk
nltk.download('stopwords')
print(stopwords.words)

stop_words = set(stopwords.words('english'))
train_data['text'] = train_data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [ ]:
#Disaster Tweets wordcloud
disaster_tweets = train_data[train_data.target == 1]

# Concatenate disaster tweets into a single string
disaster_text = []
for tweet in disaster_tweets.text:
    disaster_text.append(tweet)
disaster_text = pd.Series(disaster_text).str.cat(sep=' ')

# Generate word cloud
cloud_disaster = WordCloud(width=1600, height=800, max_font_size=200, background_color='white').generate(disaster_text)

plt.figure(figsize=(12,10))
plt.imshow(cloud_disaster, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
formal_tweets = train_data[train_data.target == 0]
formal_string = []
for t in formal_tweets.text:
    formal_string.append(t)
formal_string = pd.Series(formal_string).str.cat(sep=' ')
wordcloud = WordCloud(width=1600, height=800,max_font_size=200, background_color='white').generate(formal_string)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
#Visualizing class distribution
plt.figure(figsize=(10,5))
sns.countplot(y='target',data = train_data,palette="Paired")
plt.ylabel("Tweet problem")
plt.xlabel("# of tweets")
plt.show()

In [ ]:
#Visualizing tweet length by characaters
plt.figure(figsize=(10,5))
train_sent = train_data['text'].str.len()
sns.boxplot(x="target",y=train_sent,data=train_data,palette="Set2")
plt.xlabel("Xeet problem")
plt.ylabel("Xeet Length(char)")
plt.show()

In [ ]:
#Visualizing tweet length by words
plt.figure(figsize=(10,5))
train_sent = train_data['text'].str.split().map(lambda x : len(x))
sns.boxplot(x="target",y=train_sent,data=train_data,palette="Set1")
plt.xlabel("Tweet Fallacy")
plt.ylabel("Tweet length by word")
plt.show()

In [ ]:
train_data['word_count_train'] = train_data['text'].apply(lambda x: len(str(x).split()))
test_data['word_count_test'] = test_data['text'].apply(lambda x: len(str(x).split()))
train_data['word_count']=train_data['word_count_train']
test_data['word_count']=test_data['word_count_test']

# unique_word_count
train_data['unique_word_count_train'] = train_data['text'].apply(lambda x: len(set(str(x).split())))
test_data['unique_word_count_test'] = test_data['text'].apply(lambda x: len(set(str(x).split())))
train_data['unique_word_count']=train_data['unique_word_count_train']
test_data['unique_word_count']=test_data['unique_word_count_test']
# Stopword count
train_data['stop_word_count_train'] = train_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
test_data['stop_word_count_test'] = test_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
train_data['stop_word_count']=train_data['stop_word_count_train']
test_data['stop_word_count']=test_data['stop_word_count_test']

# url_count
train_data['url_count'] = train_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
test_data['url_count'] = test_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
train_data['mean_word_length'] = train_data['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_data['mean_word_length'] = test_data['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# char_count
train_data['char_count'] = train_data['text'].apply(lambda x: len(str(x)))
test_data['char_count'] = test_data['text'].apply(lambda x: len(str(x)))
# Unique word count


# URL count
train_data['url_count_train'] = train_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
test_data['url_count_test'] = test_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# Mean word length
train_data['mean_word_length_train'] = train_data['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_data['mean_word_length_test'] = test_data['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# Character count
train_data['char_count_train'] = train_data['text'].apply(lambda x: len(str(x)))
test_data['char_count_test'] = test_data['text'].apply(lambda x: len(str(x)))







Initialization:

The META list contains the names of the text-related features to be visualized (word_count, unique_word_count, stop_word_count, url_count, mean_word_length, char_count).
DISASTER is a boolean array indicating whether each sample in the train_data belongs to a disaster tweet (with a value of 1) or not.
Creating Subplots:

plt.subplots() creates a grid of subplots with two columns (ncols=2) and a number of rows equal to the length of META. The size of each subplot is adjusted with figsize=(20, 50) and dpi=100.
Looping Through Features:

A loop iterates over each feature in META.
For each feature, two density plots (histograms) are plotted side by side: one for disaster tweets and non-disaster tweets, and the other for the combined training and test sets.
In the left subplot (column 0), the distribution of the feature is plotted separately for disaster and non-disaster tweets using seaborn's sns.distplot().
In the right subplot (column 1), the distribution of the feature is plotted for the training and test sets combined, again using sns.distplot().
Plot Customization:

The xlabel, ylabel, and legend for each subplot are adjusted.
Titles are set for each subplot, indicating the feature name and whether it's for the training set only or for both training and test sets.
Displaying Plots:

Finally, plt.show() displays the generated subplots.
This visualization helps in understanding the distribution of different text-related features within the training data, comparing these distributions between disaster and non-disaster tweets, and also observing how these features behave across both the training and test datasets.










In [ ]:
META = ['word_count', 'unique_word_count', 'stop_word_count', 'url_count', 'mean_word_length',
                'char_count']
DISASTER = train_data['target'] == 1

fig, axes = plt.subplots(ncols=2, nrows=len(META), figsize=(20, 50), dpi=100)

for i, feature in enumerate(META):
    sns.distplot(train_data.loc[~DISASTER][feature], label='Not Disaster', ax=axes[i][0], color='green')
    sns.distplot(train_data.loc[DISASTER][feature], label='Disaster', ax=axes[i][0], color='red')

    sns.distplot(train_data[feature], label='Training', ax=axes[i][1])
    sns.distplot(test_data[feature], label='Test', ax=axes[i][1])

    for j in range(2):
        axes[i][j].set_xlabel('')
        axes[i][j].tick_params(axis='x', labelsize=12)
        axes[i][j].tick_params(axis='y', labelsize=12)
        axes[i][j].legend()

    axes[i][0].set_title(f'{feature} T Training Set', fontsize=13)
    axes[i][1].set_title(f'{feature} Training & Test Set Distribution', fontsize=13)

plt.show()

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(17, 4), dpi=100)
plt.tight_layout()

train_data.groupby('target').count()['id'].plot(kind='pie', ax=axes[0], labels=['Not Disaster (57%)', 'Disaster (43%)'])
sns.countplot(x=train_data['target'], hue=train_data['target'], ax=axes[1])

axes[0].set_ylabel('')
axes[1].set_ylabel('')
axes[1].set_xticklabels([f'Not Disaster {len(train_data["target"])}', f'Disaster {train_data["target"]}'])
axes[0].tick_params(axis='x', labelsize=15)
axes[0].tick_params(axis='y', labelsize=15)
axes[1].tick_params(axis='x', labelsize=15)
axes[1].tick_params(axis='y', labelsize=15)

axes[0].set_title('Distrubution Set', fontsize=13)
axes[1].set_title(' Training Set', fontsize=13)

plt.show()

BERT, which stands for Bidirectional Encoder Representations from Transformers, is engineered to pre-train deep bidirectional representations from unlabeled text, considering both left and right context simultaneously. This unique approach enables the pre-trained BERT model to excel across a broad spectrum of NLP tasks with minimal fine-tuning, often requiring just one additional output layer. Notably, BERT's standout feature lies in its capability to extract superior quality language features from textual data.





### LOADING THE BERT MODEL:



This bert architecture goes as follows.
Input Layer:

input_ids_custom and input_mask_custom: These are input layers defined using TensorFlow's Keras API. They are placeholders for sequences of integer tokens (such as word IDs) and attention masks (to indicate which tokens are valid and which are padding), respectively.
shape=(max_len,): Specifies the shape of input sequences, where max_len is the maximum length of input sequences.
BERT Embeddings:

embeddings = bert(input_ids_custom, attention_mask=input_mask_custom)[1]: This line applies the BERT model to the input token IDs (input_ids_custom) and attention masks (input_mask_custom).
[1] selects the pooler_output from the BERT model, which provides a representation of the entire input sequence.
Dropout Layers:

out = tf.keras.layers.Dropout(0.1)(embeddings): Dropout is applied to prevent overfitting by randomly setting a fraction of input units to zero during training. Here, a dropout rate of 0.1 is applied.
out = tf.keras.layers.Dropout(0.1)(out): Another dropout layer is added after the first one for further regularization.
Dense Layers:

out = Dense(128, activation='relu')(out): A dense layer with 128 neurons and ReLU activation function is added to the network. ReLU (Rectified Linear Unit) is a commonly used activation function that introduces non-linearity to the model.
out = Dense(32, activation='relu')(out): Another dense layer with 32 neurons and ReLU activation function follows.
Output Layer:

y = Dense(1, activation='sigmoid')(out): This is the output layer of the model. It consists of a single neuron with a sigmoid activation function, which is commonly used for binary classification tasks. The output represents the probability of the input text belonging to the positive class (e.g., disaster tweet).
Model Compilation:

model_custom = tf.keras.Model(inputs=[input_ids_custom, input_mask_custom], outputs=y): This line constructs the Keras Model, specifying the input and output layers.
model_custom.layers[2].trainable = True: This allows the BERT layer to be fine-tuned during training, enabling the model to adapt to the specific task at hand.
Overall, this architecture leverages BERT embeddings to capture contextual information from input text sequences and combines them with dense layers to perform binary classification on text data.




In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
bert = TFBertModel.from_pretrained('bert-large-uncased')

In [ ]:
tokenizer('Happy learning and keep kaggling &*&*&&')


### CONVERSION OF OUR TEXT DATA INTO BERT INPUT FORMAT:



In [ ]:
print("max len of tweets",max([len(x.split()) for x in train_data.text]))
max_length = 36


In [ ]:
x_train = tokenizer(
    text=train_data.text.tolist(),
    add_special_tokens=True,
    max_length=36,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
x_train['input_ids'].shape


In [ ]:
x_train['attention_mask'].shape


In [ ]:
y_train = train_data.target.values
y_train

In [ ]:
train_data.target.value_counts()


### BUILDING THE MODEL ARCHITECTURE:



In [ ]:
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids,attention_mask = input_mask)[0]


embeddings = bert(input_ids,attention_mask = input_mask)[1] #(0 is the last hidden states,1 means pooler_output)
# out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = tf.keras.layers.Dropout(0.1)(embeddings)

out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(1,activation = 'sigmoid')(out)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True








In [ ]:
model_custom.summary()


In [ ]:
optimizer = Adam(
    learning_rate=6e-06, # this learning rate is for bert model.
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = BinaryCrossentropy(from_logits = True)
metric = BinaryAccuracy('accuracy'),
# Compile the model
model_custom.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = metric)

In [ ]:
plot_model(model_custom, show_shapes = True)


In [ ]:
# Fit the model
final = model_custom.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
#   validation_split = 0.1,
  epochs=9,
    batch_size=10
)

Epoch 1/9
762/762 [==============================] - 215s 233ms/step - loss: 0.5084 - accuracy: 0.7644
Epoch 2/9
762/762 [==============================] - 178s 234ms/step - loss: 0.4231 - accuracy: 0.8235
Epoch 3/9
762/762 [==============================] - 178s 234ms/step - loss: 0.4058 - accuracy: 0.8382
Epoch 4/9
762/762 [==============================] - 179s 234ms/step - loss: 0.4004 - accuracy: 0.8394
Epoch 5/9
762/762 [==============================] - 179s 235ms/step - loss: 0.3897 - accuracy: 0.8400
Epoch 6/9
762/762 [==============================] - 179s 235ms/step - loss: 0.3862 - accuracy: 0.8449
Epoch 7/9
762/762 [==============================] - 179s 235ms/step - loss: 0.3809 - accuracy: 0.8501
Epoch 8/9
762/762 [==============================] - 180s 237ms/step - loss: 0.3788 - accuracy: 0.8487
Epoch 9/9
762/762 [==============================] - 179s 235ms/step - loss: 0.3695 - accuracy: 0.8550

### VISUALIZATION OF LOSS AND ACCURACY CURVE:



In [ ]:
def visualize_accuracy_and_loss(history):
    accuracy = history.history['accuracy']
    loss = history.history['loss']
    epochs = np.arange(1, len(loss) + 1)

    plt.figure(figsize=(10, 6))
    plt.plot(epochs, accuracy, 'r', label='Accuracy')
    plt.plot(epochs, loss, 'b:', label='Loss')
    plt.title('Training Accuracy and Loss Visualization')
    plt.xlabel('Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()






In [ ]:
visualize_accuracy_and_loss(test_data)


In [ ]:
x_test = tokenizer(
    text=test_data.text.tolist(),
    add_special_tokens=True,
    max_length=36,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

### PREDICTION:



In [ ]:
predicted = model_custom.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

In [ ]:
y_predicted = np.where(predicted>0.5,1,0)

In [ ]:
y_predicted = y_predicted.reshape((1,3263))[0]


In [ ]:
sample_data.head()


In [ ]:
sample_data.to_csv('submission.csv',index = False)
print(" Successfully completed! ")